In [1]:
!wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/bersatu-dengan-peradaban-5.mp3
!wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/ilusi-kuasa-1.mp3
!wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/janji-tinggal-janji-2.mp3
!wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/membina-negara-dan-bangsar-bersatu-6.mp3
!wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/orang-turk-memperjuangkan-hak-mereka-3.mp3
!wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/revolusi-barat-di-timur-4.mp3

--2020-12-14 23:53:46--  https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/bersatu-dengan-peradaban-5.mp3
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 97790704 (93M) [audio/mpeg]
Saving to: ‘bersatu-dengan-peradaban-5.mp3’

bersatu-dengan-pera 100%[===================>]  93.26M  12.9MB/s    in 8.4s    

2020-12-14 23:53:56 (11.1 MB/s) - ‘bersatu-dengan-peradaban-5.mp3’ saved [97790704/97790704]

--2020-12-14 23:53:57--  https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/ilusi-kuasa-1.mp3
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 141151220 (135M) [audio/mpeg]
Saving to: ‘ilusi-kuasa-1.mp3’

ilusi-kuasa-1

In [5]:
import os

os.system(
    'mkdir output-wav-turki output-text-turki'
)
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [6]:
import speech_recognition as sr
import soundfile as sf
from tqdm import tqdm

r = sr.Recognizer()

from pydub.silence import split_on_silence, detect_nonsilent
from pydub import AudioSegment
import numpy as np
import malaya_speech
from malaya_speech import Pipeline
from scipy.io.wavfile import write

p = Pipeline()

model_v2 = malaya_speech.vad.deep_model(model = 'vggvox-v2')
pipeline = (
    p.map(malaya_speech.utils.generator.frames, frame_duration_ms = 30)
    .batching(20)
    .foreach_map(model_v2.predict)
    .flatten()
)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [7]:
from glob import glob

mp3s = glob('*.mp3')
mp3s

['orang-turk-memperjuangkan-hak-mereka-3.mp3',
 'janji-tinggal-janji-2.mp3',
 'membina-negara-dan-bangsar-bersatu-6.mp3',
 'ilusi-kuasa-1.mp3',
 'revolusi-barat-di-timur-4.mp3',
 'bersatu-dengan-peradaban-5.mp3']

In [9]:
for file in mp3s:
    print(file)
    try:
        audio = AudioSegment.from_mp3(file)
        sample_rate = audio.frame_rate
        samples = np.array(audio.get_array_of_samples())
        samples = malaya_speech.astype.int_to_float(samples)
        samples_16k = malaya_speech.resample(samples, sample_rate, 16000)
        frames_16k = list(
            malaya_speech.utils.generator.frames(samples_16k, 30, 16000)
        )
        frames = list(
            malaya_speech.utils.generator.frames(samples, 30, sample_rate)
        )
        result = p.emit(samples_16k)
        frames_deep_v2_batch = [
            (frame, result['flatten'][no]) for no, frame in enumerate(frames)
        ]
        results = malaya_speech.split.split_vad(
            frames_deep_v2_batch, n = 2, negative_threshold = 0.1
        )

        for no in tqdm(range(len(results))):
            result = results[no]

            sf.write('test.wav', result.array, sample_rate)

            try:

                with sr.AudioFile('test.wav') as source:
                    a = r.record(source)

                text = r.recognize_google(a, language = 'ms')
                filename = f'{file}-{no}.wav'
                if len(text):
                    text_filename = (
                        f'output-text-turki/{filename}.txt'
                    )
                    with open(text_filename, 'w') as fopen:
                        fopen.write(text)

                    sf.write(
                        f'output-wav-turki/{filename}',
                        result.array,
                        sample_rate,
                    )
            except Exception as e:
                print(e)
    except Exception as e:
        print(e)

In [10]:
import soundfile as sf

lengths = []

files = glob('output-wav-turki/*.wav')
for f in tqdm(files):
    y, sr = sf.read(f)
    lengths.append(len(y) / sr)

100%|██████████| 6133/6133 [00:03<00:00, 1796.48it/s]


In [12]:
sum(lengths) / 60 / 60

7.942430782312923

In [13]:
import IPython.display as ipd
ipd.Audio(files[0])